In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
import itertools
import importlib

import adaptive_algos as aa
import helper_funcs as hf
import ObjectiveFunction as of

In [3]:
def run_experiment(model, opt_alg, start, bounds, opt_params, 
                   n_epochs, repeat=False, lr_scheduler=None):
    m = model(start=start, bounds=bounds)
    
    if opt_alg is torch.optim.Adam or opt_alg is torch.optim.SGD:
            opt_problem = of.OptimisationProblem(
                m,
                opt_alg(params=m.parameters(), **opt_params),
                n_epochs = n_epochs
            )
    else:
        opt_problem = of.OptimisationProblem(
            m,
            opt_alg(params=m.parameters(), func=m, **opt_params),
            n_epochs = n_epochs
        )
    if repeat: 
        losses, params, preds = opt_problem.run(lr_scheduler, logs=False)
    else:
        losses, params, preds = opt_problem.run(lr_scheduler, logs=True)

    if not repeat:
        fig1 = opt_problem.visualise((-bounds/2,bounds/2), (-bounds/2, bounds/2), 0.1, render="contour")
        #fig2 = px.density_heatmap(x=params[0], y=params[1], nbinsx=100, nbinsy=100)
        fig2 = hf.create_density_plot(params)
        if hasattr(opt_problem.opt, 'alpha_record'):
            fig3 = px.scatter(opt_problem.opt.alpha_record)
            return [fig1, fig2, fig3]
        
        else:
            return [fig1, fig2]
    else:
        return params[0][-1], params[1][-1]

In [5]:
importlib.reload(aa)
importlib.reload(of)
lr = 0.05
bounds = 10
height = 1.0
width = bounds/20
max_iterations = 1000
scale_annealer = lambda progress: 1 - progress

def lr_annealer1(opt, progress, lr):
    opt.param_groups[0]['lr'] = lr
lr_scheduler1 = lambda opt, progress: lr_annealer1(opt, progress, lr)

def lr_annealer2(opt, progress, lr):
    opt.param_groups[0]['lr'] = lr * (1 - progress)
lr_scheduler2 = lambda opt, progress: lr_annealer2(opt, progress, lr)

opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer, 'adjust_dir': True}
figs = run_experiment(of.AlpineN1, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                      opt_params=opt_params, n_epochs=max_iterations, 
                      lr_scheduler=lr_scheduler1  
            )
figs.extend(run_experiment(of.Ackley, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                           opt_params=opt_params, n_epochs=max_iterations, 
                           lr_scheduler=lr_scheduler2  
            ))
figs.extend(run_experiment(of.Rosenbrock, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                           opt_params=opt_params, n_epochs=max_iterations, 
                           lr_scheduler=lr_scheduler1
            ))
hf.figures_to_html(figs, 'SGD_TC_testfinal.html')

0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900


c:\Users\karls\Documents\GitHub\honours\ObjectiveFunction.py:118: RuntimeWarning:

divide by zero encountered in log



In [39]:
opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer, 'adjust_dir': True}
figs = run_experiment(of.AlpineN1, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                      opt_params=opt_params, n_epochs=max_iterations, 
                      lr_scheduler=lr_scheduler1 
            )
hf.figures_to_html(figs, 'SGD_TC_testAlpine.html')

0
100
200
300
400
500
600
700
800
900


In [46]:
opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer, 'adjust_dir': True}
figs = run_experiment(of.Ackley, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                      opt_params=opt_params, n_epochs=max_iterations,
                      lr_scheduler=lr_scheduler1
                      )
hf.figures_to_html(figs, 'SGD_TC_testAckley.html')

0
100
200
300
400
500
600
700
800
900


In [73]:
importlib.reload(of)
opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer, 'adjust_dir': True}
figs = run_experiment(of.Rosenbrock, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                      opt_params=opt_params, n_epochs=max_iterations,
                      lr_scheduler=lr_scheduler2
                      )
hf.figures_to_html(figs, 'SGD_TC_testRosenbrock.html')

0
100
200
300
400
500
600
700
800
900


c:\Users\karls\Documents\GitHub\honours\ObjectiveFunction.py:118: RuntimeWarning:

divide by zero encountered in log



In [ ]:
# compare with SGD, momentum, ADAM, SGD_TC
# percentage of runs finding global minimum, quantitiative comparisons
# time complexity comparison
# highlight the fact that it does not get trapped
# compare escape dynamics of SGD_TC with metadynamics

In [24]:
importlib.reload(aa)
importlib.reload(of)
n_exps = 1000
final_params_alpine = []
max_iterations = 1000
lr = 0.05
bounds = 10
height = 1.0
width = bounds/20

def lr_annealer2(opt, progress, lr):
    opt.param_groups[0]['lr'] = lr * (1 - progress)
lr_scheduler2 = lambda opt, progress: lr_annealer2(opt, progress, lr)
scale_annealer = lambda progress: 1 - progress
opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer}

for n in tqdm(range(n_exps)):
    final_param = run_experiment(of.AlpineN1, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                      opt_params=opt_params, n_epochs=max_iterations, repeat=True, 
                      lr_scheduler=lr_scheduler2
                      )
    final_params_alpine.append(final_param)

final_params_alpine = [list(_) for _ in list(zip(*final_params_alpine))]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [25]:
fig1 = px.density_contour(x=final_params_alpine[0], y=final_params_alpine[1])
fig1.update_traces(
    contours_coloring="fill", 
    contours_showlabels = True
)
fig1.update_layout(
    height=800,
    width=800,
    yaxis=dict(tickfont=dict(size=20)),
    xaxis=dict(tickfont=dict(size=20))
)

In [26]:
importlib.reload(aa)
importlib.reload(of)
n_exps = 1000
final_params_ackley = []
max_iterations = 1000

lr = 0.05
bounds = 10 
height = 1.0
width = bounds/20

def lr_annealer2(opt, progress, lr):
    opt.param_groups[0]['lr'] = lr * (1 - progress)
lr_scheduler2 = lambda opt, progress: lr_annealer2(opt, progress, lr)
scale_annealer = lambda progress: 1 - progress
opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer}


for n in tqdm(range(n_exps)):
    final_param = run_experiment(of.Ackley, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                      opt_params=opt_params, n_epochs=max_iterations, repeat=True
                      )
    final_params_ackley.append(final_param)
final_params_ackley = [list(_) for _ in list(zip(*final_params_ackley))]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [27]:
fig2 = px.density_contour(x=final_params_ackley[0], y=final_params_ackley[1])
fig2.update_traces(
    contours_coloring="fill", 
    contours_showlabels = True
)
fig2.update_layout(
    height=800,
    width=800,
    yaxis=dict(tickfont=dict(size=20)),
    xaxis=dict(tickfont=dict(size=20))
)

In [74]:
importlib.reload(aa)
importlib.reload(of)
n_exps = 1000
final_params_rosenbrock = []
max_iterations = 1000

lr = 0.05
bounds = 10 
height = 1.0
width = bounds/20

def lr_annealer2(opt, progress, lr):
    opt.param_groups[0]['lr'] = lr * (1 - progress)
lr_scheduler2 = lambda opt, progress: lr_annealer2(opt, progress, lr)
scale_annealer = lambda progress: 1 - progress
opt_params={'lr': lr, 'height': height, 'width': width, 'n_epochs': max_iterations, 
            'scale_annealer': scale_annealer}

for n in tqdm(range(n_exps)):
    final_param = run_experiment(of.Rosenbrock, aa.SGD_TC, start=[2.5,2], bounds=bounds,
                      opt_params=opt_params, n_epochs=max_iterations, repeat=True
                      )
    final_params_rosenbrock.append(final_param)

final_params_rosenbrock = [list(_) for _ in list(zip(*final_params_rosenbrock))]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [75]:
fig3 = px.density_contour(x=final_params_rosenbrock[0], y=final_params_rosenbrock[1],
                          nbinsx=10, nbinsy=10)
fig3.update_traces(
    contours_coloring="fill", 
    contours_showlabels = True
)
fig3.update_layout(
    height=800,
    width=800,
    yaxis=dict(tickfont=dict(size=20)),
    xaxis=dict(tickfont=dict(size=20))
)